In [1]:
import tensorflow as tf
import numpy as np

In [2]:
EPOCHS = 10

#모델 정의

In [7]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        #vgg16
        super(ConvNet,self).__init__()
        conv2d = tf.keras.layers.Conv2D
        maxpool = tf.keras.layers.MaxPool2D
        self.sequence = list()
        self.sequence.append(conv2d(16,(3,3),padding='same',activation='relu'))#출은 적어주고 입은 알아서함,28x28x1 이 들어온다. 설정값때문에 output은 28x28x16이다.
        self.sequence.append(conv2d(16,(3,3),padding='same',activation='relu'))#28x28x16
        self.sequence.append(maxpool((2,2)))#14x14x16
        
        self.sequence.append(conv2d(32,(3,3),padding='same',activation='relu'))#14x14x32
        self.sequence.append(conv2d(32,(3,3),padding='same',activation='relu'))#14x14x32
        self.sequence.append(maxpool((2,2)))#7x7x32

        self.sequence.append(conv2d(64,(3,3),padding='same',activation='relu'))#7x7x64
        self.sequence.append(conv2d(64,(3,3),padding='same',activation='relu'))#7x7x64

        self.sequence.append(tf.keras.layers.Flatten())#1568개의 vector
        self.sequence.append(tf.keras.layers.Dense(2048,activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10,activation='softmax'))#classification

    def call(self,x,training = False, mask = None):
        for layer in self.sequence:
            x = layer(x)
        return x

#루프 정의

In [4]:
@tf.function
def train_step(model,images,labels,loss_object,optimizer,train_loss,train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,predictions)
  
@tf.function
def test_step(model,images,labels,loss_object,test_loss,test_accuracy):
  predictions = model(images)

  t_loss = loss_object(labels,predictions)
  test_loss(t_loss)
  test_accuracy(labels,predictions)

#데이터셋 준비

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test,y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0 #이렇게하면 float64로 됨

x_train = x_train[...,tf.newaxis].astype(np.float32)
x_test = x_test[...,tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

11493376/11490434 [==============================] - 0s 0us/step


#학습 환경 정의

In [8]:
model = ConvNet()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

#학습

In [10]:
for epoch in range(EPOCHS):
  for images,labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_images, test_labels in test_ds:
    test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

  template = 'Epoch:{}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch:1, Loss: 0.11441505700349808, Accuracy: 96.42833709716797, Test Loss: 0.06113136559724808, Test Accuracy: 98.0199966430664
Epoch:2, Loss: 0.04354081302881241, Accuracy: 98.69999694824219, Test Loss: 0.02838991768658161, Test Accuracy: 99.08999633789062
Epoch:3, Loss: 0.034109704196453094, Accuracy: 98.94833374023438, Test Loss: 0.03759456053376198, Test Accuracy: 98.79999542236328
Epoch:4, Loss: 0.027489937841892242, Accuracy: 99.14500427246094, Test Loss: 0.03966477885842323, Test Accuracy: 98.93000030517578
Epoch:5, Loss: 0.021818311884999275, Accuracy: 99.33666229248047, Test Loss: 0.04533319175243378, Test Accuracy: 98.77999877929688
Epoch:6, Loss: 0.020272208377718925, Accuracy: 99.39666748046875, Test Loss: 0.037856314331293106, Test Accuracy: 99.1500015258789
Epoch:7, Loss: 0.017730573192238808, Accuracy: 99.44166564941406, Test Loss: 0.03292794153094292, Test Accuracy: 99.11000061035156
Epoch:8, Loss: 0.01509073656052351, Accuracy: 99.56832885742188, Test Loss: 0.03419644